In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download clmentbisaillon/fake-and-real-news-dataset

fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip fake-and-real-news-dataset.zip

Archive:  fake-and-real-news-dataset.zip
replace Fake.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


**Importing Libraries**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import spacy
import wordcloud
import pandas as pd
import os
import re
import tensorflow as tf
from tqdm import tqdm
import random
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.activations import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

In [ ]:
fake_df = pd.read_csv('/content/Fake.csv')
fake_df['class'] = 0
true_df = pd.read_csv('/content/True.csv')
true_df['class'] = 1

In [ ]:
news_data = pd.concat([fake_df, true_df], axis=0, join="inner")
news_data = news_data.sample(frac=1).reset_index(drop=True)
news_data.head()

,title,text,subject,date,class
0,This Letter From The Fed Warns Of The Next Gr...,In case you weren t paying attention: Just as ...,News,"April 16, 2016",0
1,Charlottesville: Far Left vs Far Right Clashes...,"21st Century WireYesterday, violence broke out...",Middle-east,"August 13, 2017",0
2,COPYCAT MUSLIM TERRORIST ARRESTED With Assault...,Assimilation fail A French Muslim has been arr...,left-news,"Mar 23, 2017",0
3,"Exclusive: Obama, aides expected to weigh Syri...",WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"October 13, 2016",1
4,"Pope urges Trump to show concern for poor, be ...",VATICAN CITY (Reuters) - Pope Francis urged U....,politicsNews,"January 20, 2017",1


In [ ]:
news_data.drop('date', axis = 1, inplace = True)

In [ ]:
news_data.head()

,title,text,subject,class
0,This Letter From The Fed Warns Of The Next Gr...,In case you weren t paying attention: Just as ...,News,0
1,Charlottesville: Far Left vs Far Right Clashes...,"21st Century WireYesterday, violence broke out...",Middle-east,0
2,COPYCAT MUSLIM TERRORIST ARRESTED With Assault...,Assimilation fail A French Muslim has been arr...,left-news,0
3,"Exclusive: Obama, aides expected to weigh Syri...",WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,1
4,"Pope urges Trump to show concern for poor, be ...",VATICAN CITY (Reuters) - Pope Francis urged U....,politicsNews,1


In [ ]:
news_data ['content'] = news_data['title'] +': ' + news_data['text']
news_data.drop(['title','text', 'subject'], axis = 1, inplace = True)
news_data.head()

,class,content
0,0,This Letter From The Fed Warns Of The Next Gr...
1,0,Charlottesville: Far Left vs Far Right Clashes...
2,0,COPYCAT MUSLIM TERRORIST ARRESTED With Assault...
3,1,"Exclusive: Obama, aides expected to weigh Syri..."
4,1,"Pope urges Trump to show concern for poor, be ..."


In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels=train_test_split(news_data['content'].to_numpy(),
                                                                            news_data['class'].to_numpy(),
                                                                            test_size=0.2,
                                                                            random_state=42)

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 418


from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode='int',
                                   output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                            output_dim=128,
                            embeddings_initializer='uniform',
                            input_length=max_length)

In [ ]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype='string')

# Pass inputs to text_vectorizer(convert text into numbers)
x = text_vectorizer(inputs) 

# Convert text_vectorizer layer into embedding layer
x = embedding(x)

# Model 
x = layers.LSTM(64)(x)


# output
outputs = layers.Dense(1, activation='sigmoid')(x)

# Pass inputs and outputs to our model
model = tf.keras.Model(inputs, outputs, name='model_LSTM')

In [ ]:
model.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 418)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 418, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
______________________________________________

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
model_history = model.fit(train_sentences,
                          train_labels,
                          epochs=50,
                          validation_data=(val_sentences, val_labels),
                          callbacks=[callback]
                         )

Epoch 1/50
1123/1123 [==============================] - 113s 97ms/step - loss: 0.3560 - accuracy: 0.8514 - val_loss: 0.1877 - val_accuracy: 0.9484
Epoch 2/50
1123/1123 [==============================] - 107s 95ms/step - loss: 0.2495 - accuracy: 0.9123 - val_loss: 0.2632 - val_accuracy: 0.9160
Epoch 3/50
1123/1123 [==============================] - 107s 95ms/step - loss: 0.3545 - accuracy: 0.8439 - val_loss: 0.3244 - val_accuracy: 0.8790
Epoch 4/50
1123/1123 [==============================] - 106s 94ms/step - loss: 0.3115 - accuracy: 0.8820 - val_loss: 0.4800 - val_accuracy: 0.8595
Epoch 5/50
1123/1123 [==============================] - 106s 95ms/step - loss: 0.4263 - accuracy: 0.8152 - val_loss: 0.1644 - val_accuracy: 0.9487
Epoch 6/50
1123/1123 [==============================] - 106s 94ms/step - loss: 0.1920 - accuracy: 0.9382 - val_loss: 0.0434 - val_accuracy: 0.9912
Epoch 7/50
1123/1123 [==============================] - 105s 94ms/step - loss: 0.0302 - accuracy: 0.9928 - val_loss: 0